In [53]:
import pandas as pd
import numpy as np
import altair as alt
import eco_style
import os
alt.themes.enable("light")  

ThemeRegistry.enable('light')

In [75]:
df = pd.read_csv("clarke_2003_britain_prices.csv", sep=" ", encoding="utf-8")
df

,Year,Arable,Meat,Dairy,Wool,Pasture,Wood_Index,Farm_Index
0,1770,64.0,57.6,50.6,29.3,51.3,89.1,58.2
1,1771,75.9,60.2,50.8,32.5,53.6,85.4,65.0
2,1772,82.3,63.4,56.2,30.5,57.4,91.8,70.1
3,1773,86.1,63.8,56.6,31.9,58.7,101.4,72.7
4,1774,86.1,61.8,56.6,34.6,57.0,96.0,71.7
...,...,...,...,...,...,...,...,...
75,1845,95.0,73.2,92.4,67.1,80.3,119.7,88.3
76,1846,96.6,86.5,96.8,59.8,85.3,112.0,91.3
77,1847,125.2,96.2,97.8,55.9,88.5,101.4,106.6
78,1848,87.6,93.2,91.6,51.3,84.6,94.7,86.1


In [82]:
# Index each Commodity's 1793 price to 100
#df = pd.read_csv("grain_prices.tsv", sep=" ")
df = pd.read_csv("clarke_2003_britain_prices.csv", sep=" ", encoding="utf-8")
df = df.melt(id_vars=['Year'], var_name='Commodity', value_name='value')
df_1793 = df[df['Year'] == 1793].set_index('Commodity')['value']
df = df.merge(df_1793, on='Commodity', suffixes=('', '_1793'))
df['indexed_value'] = df['value'] / df['value_1793'] * 100
df = df.dropna(subset=['indexed_value'])
df['Year'] = pd.to_datetime(df['Year'], format='%Y')

#df = df[df.Commodity.isin(['Wheat', 'Oats', 'Rye'])]
df =  df[df.Commodity.isin(['Farm_Index'])]


events = [
    {"Year": pd.to_datetime("1793"), "Label": "War begins"},
    # {"Year": pd.to_datetime("1795"), "Label": "Food riots"},
    {"Year": pd.to_datetime("1800"), "Label": "Famine"},
    {"Year": pd.to_datetime("1805"), "Label": "Trafalgar"},
    # {"Year": pd.to_datetime("1806"), "Label": "Continental System"},
    # {"Year": pd.to_datetime("1810"), "Label": "Price peak"},
    {"Year": pd.to_datetime("1815"), "Label": "Waterloo"},
    # {"Year": pd.to_datetime("1816"), "Label": "Year Without\nSummer"},
    # {"Year": pd.to_datetime("1819"), "Label": "Gold Standard debate"}
]

events = pd.DataFrame(events)
events['Label'] = events['Label'].apply(lambda x: x.split('\n'))  

rules = alt.Chart(events).mark_rule(color='rgba(0,0,0,0.5)', strokeDash=[3,5]).encode(
    x='Year:T',
    tooltip='Label:N'
)

text = alt.Chart(events).mark_text(align='left', dx=0, dy=-5, 
                                   baseline='bottom',
                                   angle=0).encode(
    x='Year:T',
    y=alt.value(0),  # Adjust vertical position
    text='Label:N'
)

line = alt.Chart(df).mark_line(
    interpolate='monotone',
    color="#36b7b4"
).encode(
    x=alt.X("Year:T", title=""),
    y=alt.Y("indexed_value:Q",
            axis=alt.Axis(
                labelExpr="(datum.value == 100 ? '1793=100' : datum.label)",
            ),
             title="Farm Price Index"),
    # color=alt.Color("Commodity:N", title="Commodity", scale=alt.Scale()),
)

chart = (line + rules + text).properties(width=700, height=400)
chart.save("clarke_2003_britain_prices.png", scale_factor=2)
chart.save("clarke_2003_britain_prices.json")
chart

alt.LayerChart(...)

In [68]:
df.query("Year == '1793-01-01'")

/var/folders/07/g_yq2xsn0mvb4xnyvr18jjmw0000gn/T/ipykernel_40988/311782109.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df.query("Year == '1793-01-01'")


,Year,Commodity,value,value_1793,indexed_value
23,1793-01-01,Wheat,5.98,5.98,100.0
103,1793-01-01,Rye,4.52,4.52,100.0
183,1793-01-01,Barley,3.95,3.95,100.0
263,1793-01-01,Oats,2.61,2.61,100.0
343,1793-01-01,Peas,4.72,4.72,100.0
423,1793-01-01,Beans,4.18,4.18,100.0
503,1793-01-01,Potato,1.79,1.79,100.0
583,1793-01-01,Hops,109.88,109.88,100.0
